In [54]:
import pandas as pd

In [55]:
df = pd.read_csv("Data/cleaned_dataframe.csv", low_memory = False)

In [56]:
df.head(2)

,Unnamed: 0,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Hearing Result,Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location
0,22,0200788327,2019-02-08,POLICE DEPARTMENT,ROBERTO,RIVERA,BRONX,2535.0,21.0,1375,OGDEN AVENUE,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,1,NaN,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 1045...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK
1,23,0200786576,2020-02-09,POLICE DEPARTMENT,MOHAMED J,ALABDI,BRONX,3191.0,25.0,2630,JEROME AVENUE,BRONX,10468,NEW YORK,BRONX,1129,WHEELER AVENUE,BRONX,10472,NEW YORK,2,SAU: MANH,0.0,0.0,AD6L,AC 19-190 B,"RIGHT OF WAY - FAILURE TO YIELD, PHYSICAL INJURY",250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3191.0 25.0 2630 JEROME AVENUE BRONX 104...,BRONX 1129 WHEELER AVENUE BRONX 10472 NEW YORK


In [57]:
df.drop("Unnamed: 0", axis=1)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Hearing Result,Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location
0,0200788327,2019-02-08,POLICE DEPARTMENT,ROBERTO,RIVERA,BRONX,2535.0,21.0,1375,OGDEN AVENUE,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,1,NaN,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 1045...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK
1,0200786576,2020-02-09,POLICE DEPARTMENT,MOHAMED J,ALABDI,BRONX,3191.0,25.0,2630,JEROME AVENUE,BRONX,10468,NEW YORK,BRONX,1129,WHEELER AVENUE,BRONX,10472,NEW YORK,2,SAU: MANH,0.0,0.0,AD6L,AC 19-190 B,"RIGHT OF WAY - FAILURE TO YIELD, PHYSICAL INJURY",250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3191.0 25.0 2630 JEROME AVENUE BRONX 104...,BRONX 1129 WHEELER AVENUE BRONX 10472 NEW YORK
2,0201486039,2020-02-07,POLICE DEPARTMENT,SEGUN D,OGUNDEINDE,BROOKLYN,1971.0,27.0,1102,BEDFORD AVENUE,BROOKLYN,11216,NEW YORK,BROOKLYN,1170,NOSTRAND AVENUE,BROOKLYN,11225,NEW YORK,2,SAU: MANH,0.0,0.0,AD6L,AC 19-190 B,"RIGHT OF WAY - FAILURE TO YIELD, PHYSICAL INJURY",250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN 1971.0 27.0 1102 BEDFORD AVENUE BROOK...,BROOKLYN 1170 NOSTRAND AVENUE BROOKLYN 11225 N...
3,0201641440,2020-02-08,POLICE DEPARTMENT,EMMANUEL,BARRIENTOS,BRONX,3293.0,52.0,274,EAST 194 STREET,BRONX,10458,NEW YORK,BRONX,2557,MARION AVENUE,BRONX,10458,NEW YORK,1,NaN,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3293.0 52.0 274 EAST 194 STREET BRONX 10...,BRONX 2557 MARION AVENUE BRONX 10458 NEW YORK
4,0201648096,2020-02-02,POLICE DEPARTMENT,RICARDO,FLORES,BRONX,3738.0,43.0,1177,WHEELER AVENUE,BRONX,10472,NEW YORK,BRONX,1217,BOYNTON AVENUE,BRONX,10472,NEW YORK,1,NaN,112.0,0.0,AS9I,16-118 6,PUBLIC URINATION 1ST OFFENSE,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3738.0 43.0 1177 WHEELER AVENUE BRONX 10...,BRONX 1217 BOYNTON AVENUE BRONX 10472 NEW YORK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284320,0127956804,2002-03-23,SANITATION POLICE,NaN,NaN,STATEN IS,1155.0,1.0,225,WALKER STREET,STATEN IS,10303,NEW YORK,STATEN IS,225,WALKER STREET,STATEN IS,10303,NEW YORK,0,NaN,300.0,NaN,AS06,16-118 2,DIRTY SIDEWALK,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STATEN IS 1155.0 1.0 225 WALKER STREET STATEN ...,STATEN IS 225 WALKER STREET STATEN IS 10303 NE...
284321,0139928077,2004-05-25,SEAGATE POLICE,NaN,NaN,BROOKLYN,6999.0,32.0,4018,SEA GATE AVENUE,BROOKLYN,11224,NEW YORK,BROOKLYN,4018,SEA GATE AVENUE,BROOKLYN,11224,NEW YORK,0,NaN,300.0,NaN,AS07,16-118(2),SIDEWALK OBSTRUCTION,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN 6999.0 32.0 4018 SEA GATE AVENUE BROO...,BROOKLYN 4018 SEA GATE AVENUE BROOKLYN 11224 N...
284322,0139928407,2004-09-15,SEAGATE POLICE,NaN,NaN,BROOKLYN,7028.0,25.0,4022,SURF AVENUE,BROOKLYN,11224,NEW YORK,BROOKLYN,4022,SURF AVENUE,BROOKLYN,11224,NEW YORK,0,NaN,300.0,NaN,AS06,16-118(2),DIRTY SIDEWALK,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN 7028.0 25.0 4022 SURF AVENUE BROOKLYN...,BROOKLYN 

In [8]:
df['Violation Location (Zip Code)'].value_counts()

10036    12883
10001     9004
11207     8105
10466     7250
11368     6362
         ...  
11304        1
11307        1
10174        1
11543        1
10168        1
Name: Violation Location (Zip Code), Length: 238, dtype: int64

In [9]:
df["Respondent Address (Zip Code)"].value_counts()

11207    8549
10466    7363
11368    7322
10457    6801
10456    6516
         ... 
10146       1
7506        1
11604       1
11939       1
14212       1
Name: Respondent Address (Zip Code), Length: 1250, dtype: int64

In [10]:
zip_code = pd.read_csv("Data/Demographic_Statistics_By_Zip_Code.csv", low_memory = False)

In [11]:
zip_code.loc[zip_code["JURISDICTION NAME"] == 12783]

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
229,12783,201,66,0.33,135,0.67,0,0,201,100,...,201,100,77,0.38,124,0.62,0,0,201,100


In [12]:
merged_df = pd.merge(df, zip_code, left_on = "Respondent Address (Zip Code)", right_on = "JURISDICTION NAME", how="inner")

In [13]:
merged_df = merged_df.drop("Unnamed: 0", axis=1)

In [14]:
merged_df['Ticket Number'].is_unique

True

In [15]:
pd.set_option('display.max_columns', 2000000000)
merged_df.iloc[277360:]

,Ticket Number,Violation Date,Issuing Agency,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (Floor),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Hearing Result,Scheduled Hearing Location,Decision Location (Borough),Total Violation Amount,Violation Details,Penalty Imposed,Paid Amount,Violation Description,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,Charge #4: Code,Charge #4: Code Section,Charge #4: Code Description,Charge #4: Infraction Amount,Charge #5: Code,Charge #5: Code Section,Charge #5: Code Description,Charge #5: Infraction Amount,Charge #6: Code,Charge #6: Code Section,Charge #6: Code Description,Charge #6: Infraction Amount,Charge #7: Code,Charge #7: Code Section,Charge #7: Code Description,Charge #7: Infraction Amount,Charge #8: Code,Charge #8: Code Section,Charge #8: Code Description,Charge #8: Infraction Amount,Charge #9: Code,Charge #9: Code Section,Charge #9: Code Description,Charge #9: Infraction Amount,Charge #10: Code,Charge #10: Code Section,Charge #10: Code Description,Charge #10: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
277360,0101267789,05/02/1999,POLICE DEPARTMENT,QUEENS,NaN,NaN,123-01,NaN,NaN,QUEENS,11368,NEW YORK,NOT NYC,NaN,DAYTOP VILL PO BOX 340,SWAN LAKE,12783,NEW YORK,WRITTEN OFF,MANHATTAN,NaN,250.0,NaN,1000.0,0.0,NaN,AG01,A.C. 20-453,UNLICENSED GENERAL VENDOR,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QUEENS 123-01 QUEENS 11368 NEW YORK,NOT NYC DAYTOP VILL PO BOX 340 SWAN LAKE 1278...,12783,201,66,0.33,135,0.67,0,0,201,100,0,0.0,0,0.0,2,0.01,0,0.0,195,0.97,0,0.0,3,0.01,1,0.0,201,99,4,0.02,197,0.98,0,0.0,0,0,201,100,77,0.38,124,0.62,0,0,201,100


In [39]:
column_list = list(merged_df.columns)
column_list

['Hearing Result',
 'Ticket Number',
 'Violation Date',
 'Issuing Agency',
 'Violation Location (Borough)',
 'Violation Location (Block No.)',
 'Violation Location (Lot No.)',
 'Violation Location (House #)',
 'Violation Location (Street Name)',
 'Violation Location (Floor)',
 'Violation Location (City)',
 'Violation Location (Zip Code)',
 'Violation Location (State Name)',
 'Respondent Address (Borough)',
 'Respondent Address (House #)',
 'Respondent Address (Street Name)',
 'Respondent Address (City)',
 'Respondent Address (Zip Code)',
 'Respondent Address (State Name)',
 'Scheduled Hearing Location',
 'Decision Location (Borough)',
 'Total Violation Amount',
 'Violation Details',
 'Penalty Imposed',
 'Paid Amount',
 'Violation Description',
 'Charge #1: Code',
 'Charge #1: Code Section',
 'Charge #1: Code Description',
 'Charge #1: Infraction Amount',
 'Charge #2: Code',
 'Charge #2: Code Section',
 'Charge #2: Code Description',
 'Charge #2: Infraction Amount',
 'Charge #3: Code',


In [17]:
first_column = merged_df.pop('Hearing Result')
  
# insert column using insert(position,column_name,first_column) function
merged_df.insert(0, 'Hearing Result', first_column)

In [18]:
merged_df

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (Floor),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Scheduled Hearing Location,Decision Location (Borough),Total Violation Amount,Violation Details,Penalty Imposed,Paid Amount,Violation Description,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,Charge #4: Code,Charge #4: Code Section,Charge #4: Code Description,Charge #4: Infraction Amount,Charge #5: Code,Charge #5: Code Section,Charge #5: Code Description,Charge #5: Infraction Amount,Charge #6: Code,Charge #6: Code Section,Charge #6: Code Description,Charge #6: Infraction Amount,Charge #7: Code,Charge #7: Code Section,Charge #7: Code Description,Charge #7: Infraction Amount,Charge #8: Code,Charge #8: Code Section,Charge #8: Code Description,Charge #8: Infraction Amount,Charge #9: Code,Charge #9: Code Section,Charge #9: Code Description,Charge #9: Infraction Amount,Charge #10: Code,Charge #10: Code Section,Charge #10: Code Description,Charge #10: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,DEFAULTED,0200788327,02/08/2019,POLICE DEPARTMENT,BRONX,2535.0,21.0,1375,OGDEN AVENUE,NaN,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20200325N,25.0,0.0,NaN,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 104...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
1,DISMISSED,046611776R,02/15/2020,SANITATION POLICE,BRONX,2838.0,12.0,1675,GRAND CONCOURSE,NaN,BRONX,10452,NEW YORK,BRONX,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,SAU: MANH,BY PHONE,100.0,20200324N,0.0,0.0,NaN,AS06,A.C. 16-118 2 A,DIRTY SIDEWALK DIRTY AREA,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2838.0 12.0 1675 GRAND CONCOURSE BRONX ...,BRONX 1675 GRAND CONCOURSE BRONX 10452 NEW YORK,1045

In [19]:
merged_df["Charge #10: Infraction Amount"].isnull().value_counts()

True    277361
Name: Charge #10: Infraction Amount, dtype: int64

In [20]:
def drop_columns(dataframe):
    
    for x in column_list:
        if dataframe[x].isnull().all() == True:
            
            dataframe = dataframe.drop(x, axis = 1)
    
    return dataframe 
    

In [21]:
drop_columns(merged_df)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Scheduled Hearing Location,Decision Location (Borough),Total Violation Amount,Violation Details,Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,DEFAULTED,0200788327,02/08/2019,POLICE DEPARTMENT,BRONX,2535.0,21.0,1375,OGDEN AVENUE,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20200325N,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 104...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
1,DISMISSED,046611776R,02/15/2020,SANITATION POLICE,BRONX,2838.0,12.0,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,BRONX,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,SAU: MANH,BY PHONE,100.0,20200324N,0.0,0.0,AS06,A.C. 16-118 2 A,DIRTY SIDEWALK DIRTY AREA,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2838.0 12.0 1675 GRAND CONCOURSE BRONX ...,BRONX 1675 GRAND CONCOURSE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
2,DEFAULTED,0200404088,07/02/2018,POLICE DEPARTMENT,BRONX,3076.0,9.0,2428,HUGHES AVENUE,BRONX,10458,NEW YORK,BRONX,134,ELLIT PLACE,BRONX,10452,NEW YORK,BRONX,NaN,75.0,20180820N,112.0,0.0,AS9I,16-118 6,PUBLIC URINATION 1ST OFFENSE,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3076.0 9.0 2428 HUGHES AVENUE BRONX 104...,BRONX 134 ELLIT PLACE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
3,DEFAULTED,0200430780,08/12/2018,POLICE DEPARTMENT,BRONX,2510.0,40.0,1195,ANDERSON AVENUE,BRONX,10452,NEW YORK,BRONX,1195,ANDERSON AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20181009Y,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2510.0

In [22]:
merged_df.head(3)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (Floor),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Scheduled Hearing Location,Decision Location (Borough),Total Violation Amount,Violation Details,Penalty Imposed,Paid Amount,Violation Description,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,Charge #4: Code,Charge #4: Code Section,Charge #4: Code Description,Charge #4: Infraction Amount,Charge #5: Code,Charge #5: Code Section,Charge #5: Code Description,Charge #5: Infraction Amount,Charge #6: Code,Charge #6: Code Section,Charge #6: Code Description,Charge #6: Infraction Amount,Charge #7: Code,Charge #7: Code Section,Charge #7: Code Description,Charge #7: Infraction Amount,Charge #8: Code,Charge #8: Code Section,Charge #8: Code Description,Charge #8: Infraction Amount,Charge #9: Code,Charge #9: Code Section,Charge #9: Code Description,Charge #9: Infraction Amount,Charge #10: Code,Charge #10: Code Section,Charge #10: Code Description,Charge #10: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,DEFAULTED,0200788327,02/08/2019,POLICE DEPARTMENT,BRONX,2535.0,21.0,1375,OGDEN AVENUE,NaN,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20200325N,25.0,0.0,NaN,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 104...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.0,0,0.0,0,0.0,5,0.29,1,0.06,1,0.06,17,100,0,0.0,17,1.0,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
1,DISMISSED,046611776R,02/15/2020,SANITATION POLICE,BRONX,2838.0,12.0,1675,GRAND CONCOURSE,NaN,BRONX,10452,NEW YORK,BRONX,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,SAU: MANH,BY PHONE,100.0,20200324N,0.0,0.0,NaN,AS06,A.C. 16-118 2 A,DIRTY SIDEWALK DIRTY AREA,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2838.0 12.0 1675 GRAND CONCOURSE BRONX ...,BRONX 1675 GRAND CONCOURSE BRONX 10452 NEW YORK,10452,17

In [46]:
# check for columns that are completely empty and drop them from the dataframe

empty_columns = []

for x in column_list:
    
    if merged_df[x].isnull().all() == True:
        empty_columns.append(x)

In [48]:
empty_columns

['Violation Location (Floor)',
 'Violation Description',
 'Charge #4: Code',
 'Charge #4: Code Section',
 'Charge #4: Code Description',
 'Charge #4: Infraction Amount',
 'Charge #5: Code',
 'Charge #5: Code Section',
 'Charge #5: Code Description',
 'Charge #5: Infraction Amount',
 'Charge #6: Code',
 'Charge #6: Code Section',
 'Charge #6: Code Description',
 'Charge #6: Infraction Amount',
 'Charge #7: Code',
 'Charge #7: Code Section',
 'Charge #7: Code Description',
 'Charge #7: Infraction Amount',
 'Charge #8: Code',
 'Charge #8: Code Section',
 'Charge #8: Code Description',
 'Charge #8: Infraction Amount',
 'Charge #9: Code',
 'Charge #9: Code Section',
 'Charge #9: Code Description',
 'Charge #9: Infraction Amount',
 'Charge #10: Code',
 'Charge #10: Code Section',
 'Charge #10: Code Description',
 'Charge #10: Infraction Amount']

In [49]:
merged_df.drop(empty_columns, axis = 1, inplace=True)

In [50]:
merged_df

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Scheduled Hearing Location,Decision Location (Borough),Total Violation Amount,Violation Details,Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,DEFAULTED,0200788327,02/08/2019,POLICE DEPARTMENT,BRONX,2535.0,21.0,1375,OGDEN AVENUE,BRONX,10452,NEW YORK,BRONX,1014,GERARD AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20200325N,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2535.0 21.0 1375 OGDEN AVENUE BRONX 104...,BRONX 1014 GERARD AVENUE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
1,DISMISSED,046611776R,02/15/2020,SANITATION POLICE,BRONX,2838.0,12.0,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,BRONX,1675,GRAND CONCOURSE,BRONX,10452,NEW YORK,SAU: MANH,BY PHONE,100.0,20200324N,0.0,0.0,AS06,A.C. 16-118 2 A,DIRTY SIDEWALK DIRTY AREA,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2838.0 12.0 1675 GRAND CONCOURSE BRONX ...,BRONX 1675 GRAND CONCOURSE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
2,DEFAULTED,0200404088,07/02/2018,POLICE DEPARTMENT,BRONX,3076.0,9.0,2428,HUGHES AVENUE,BRONX,10458,NEW YORK,BRONX,134,ELLIT PLACE,BRONX,10452,NEW YORK,BRONX,NaN,75.0,20180820N,112.0,0.0,AS9I,16-118 6,PUBLIC URINATION 1ST OFFENSE,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 3076.0 9.0 2428 HUGHES AVENUE BRONX 104...,BRONX 134 ELLIT PLACE BRONX 10452 NEW YORK,10452,17,12,0.71,5,0.29,0,0,17,100,0,0.0,10,0.59,0,0.00,0,0.0,0,0.00,5,0.29,1,0.06,1,0.06,17,100,0,0.00,17,1.00,0,0.0,0,0,17,100,3,0.18,14,0.82,0,0,17,100
3,DEFAULTED,0200430780,08/12/2018,POLICE DEPARTMENT,BRONX,2510.0,40.0,1195,ANDERSON AVENUE,BRONX,10452,NEW YORK,BRONX,1195,ANDERSON AVENUE,BRONX,10452,NEW YORK,BRONX,NaN,25.0,20181009Y,25.0,0.0,AX25,AC 10-125,OPEN CONTAINER CONSUMPTION OF ALCOHOL ON STREETS,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRONX 2510.0